<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>5. Proyecto de clasificación multiclase</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Importar clases, funciones y conjunto de datos](#section1)
* [2. Codificar la variable de salida](#section2)
* [3. Definir la red neuronal](#section3)
* [4. Evaluar el modelo](#section4)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En esta lección vamos a desarrollar y evaluar modelos de redes neuronales para problemas de clasificación multiclase. Así, veremos
* Cómo preparar datos de clasificación multiclase para modelar con redes neuronales.
* Cómo evaluar modelos de redes neuronales de Keras con scikit-learn.

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Importar clases, funciones y conjunto de datos</font>

Podemos comenzar importando todas las clases, funciones y dataset que necesitaremos en este tutorial. 

In [3]:
# Multiclass Classification with the Iris Flowers Dataset
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

# load dataset
dataframe = pd.read_csv("https://raw.githubusercontent.com/FMunyoz/colab/main/Datasets/iris.csv", header = None)
dataset = dataframe.values
X = dataset[:, 0:4].astype(float)
y = dataset[:, 4]

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versic

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Iris](https://archive.ics.uci.edu/ml/datasets/Iris)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Codificar la variable de salida</font>

Como tenemos un problema de clasificación multiclase recordemos que debemos de utilizar One-Hot Encoding para poder formatear la salida. 

Por ejemplo, en este problema los tres valores de clase:
```
    Iris-setosa
    Iris-versicolor
    Iris-virginica
```

Por lo que convertimos esta salida en una codificación binaría como:
```
    Iris-setosa, Iris-versicolor, Iris-virginica
        1,             0,              0
        0,             1,              0
        0,             0,              1
```

Por tanto, 
1. Codificando primero las cadenas de manera coherente en números enteros utilizando la clase `LabelEncoder` de scikit-learn. 
2. Luego, conviertimos el vector de números enteros en One-Hot Enconding usando la función de Keras `to_categorical()`.

In [5]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
print(encoded_y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y  = np_utils.to_categorical(encoded_y)
print(dummy_y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Definir la red neuronal</font>

A continuación se muestra el procedimiento a la hora de crear la función a trabajar:
1. Crea una red simple completamente conectada con una capa oculta de 8 neuronas. 
2. La capa oculta utiliza una función de activación ReLu. 
3. Debido a que utilizamos One-Hot Encoding, la capa de salida debe crear 3 valores de salida, uno para cada clase. 
4. El valor de salida con el valor más grande se tomará como la clase predicha por el modelo. La topología quedaría así:
```
    4 inputs -> [8 hidden nodes] -> 3 outputs
```
5. Tendremos una función de activación Softmax en la capa de salida. 
6. Finalmente, la red utiliza Adam con una función de pérdida logarítmica (`categorical_crossentropy`).

In [10]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation = 'relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



Ahora podemos crear nuestro `KerasClassifier` para usarlo en scikit-learn. 

Aquí, pasamos el número de épocas como 200 y el tamaño de batch como 5 para usar al entrenar el modelo. 

In [11]:
estimator = KerasClassifier(build_fn=baseline_model, epochs = 200, batch_size=5, verbose=0)

<a id="section4"></a>
# <font color="#004D7F" size=6>4. Evaluar el modelo</font>

Ahora podemos evaluar nuestro modelo _(estimator)_ en nuestro conjunto de datos (`X` e `dummy_y`). Utilizando un procedimiento de validación cruzada de 10 veces (`kfold`)

In [12]:
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 97.33% (4.42%)


<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>